# Dataset

In [1]:
import sys

# add the parent directory to the path so we can import the module
sys.path.append("/data2/eranario/scratch/strawberry-yield-forecasting")

In [2]:
import torch
from src.dataset import StrawberryDataset

In [3]:
# device is cuda else cpu
device = "cuda" if torch.cuda.is_available() else "cpu"

In [19]:
path_to_counts = "/data2/eranario/data/Strawberry-Yield-Forecasting/"
path_to_weights = "/data2/eranario/data/Strawberry-Yield-Forecasting/weights/weights.csv"
n_seq = 5
seq_l = 3
n_folds = 2
k_fold = 2
dataset = StrawberryDataset(path_to_counts, path_to_weights, k_fold=k_fold,
                            n_seq=n_seq, seq_l=seq_l, n_folds=n_folds)

In [20]:
# Check date parsing
months, days, years = dataset.months, dataset.days, dataset.years
print("\nParsed Dates:")
print("Months:", months)
print("Days:", days)
print("Years:", years)


X_data, y_data = dataset.fnX, dataset.fny
print("\nOrganized Data Shapes:")
print("X_data shape:", X_data.shape)  # Expected: (num_samples, num_features)
print("y_data shape:", y_data.shape)  # Expected: (num_samples, num_labels)

# Display sample data from X and y
print("\nSample X_data:", X_data[0][:10])  # Display first 10 features of first sample
print("Sample y_data:", y_data[0])         # Display first sample of y_data

print("\nDataset length (number of samples):", len(dataset))

X_sample, y_sample = dataset[0]
print("\nSample from __getitem__:")
print("X_sample:", X_sample[:10])  # Display first 10 features of X_sample
print("y_sample:", y_sample)


Parsed Dates:
Months: ['06', '06', '07', '07', '07', '07', '07', '07', '08', '08', '08', '08', '08', '09', '09', '09']
Days: ['17', '28', '05', '08', '15', '19', '26', '29', '02', '05', '09', '12', '29', '01', '20', '22']
Years: ['2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022']

Organized Data Shapes:
X_data shape: (495, 3, 7)
y_data shape: (495, 1)

Sample X_data: [[0.         0.25957873 0.08863333 0.         0.         0.56364411
  1.        ]
 [0.         0.         0.         0.         0.         0.
  0.47619048]
 [0.         0.         0.         0.3789208  0.         0.34544651
  0.14285714]]
Sample y_data: [0.33320494]

Dataset length (number of samples): 495

Sample from __getitem__:
X_sample: tensor([[0.0000, 0.2596, 0.0886, 0.0000, 0.0000, 0.5636, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4762],
        [0.0000, 0.0000, 0.0000, 0.3789, 0.0000, 0.3454, 0.1429]])
y_sample:

# Dataloader

In [21]:
import torch
from torch.utils.data import DataLoader

In [22]:
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [23]:
len(train_loader), len(val_loader)

(371, 124)

In [24]:
# try to get a batch of data
for i, (X_batch, y_batch) in enumerate(train_loader):
    print("\nBatch", i)
    print("X_batch shape:", X_batch.shape)
    print("y_batch shape:", y_batch.shape)
    print("X_batch:", X_batch)
    print("y_batch:", y_batch)
    break


Batch 0
X_batch shape: torch.Size([1, 3, 7])
y_batch shape: torch.Size([1, 1])
X_batch: tensor([[[0.0000, 0.1742, 0.3561, 0.0000, 0.0000, 0.7454, 1.0000],
         [0.0000, 0.0000, 0.2494, 0.1078, 0.0000, 0.3828, 0.5882],
         [0.0000, 0.0000, 0.1389, 0.3002, 0.0000, 0.3292, 0.4118]]])
y_batch: tensor([[0.6282]])


# Training

In [25]:
from torch.optim import Adam
from torch.nn import MSELoss
from src.model import LSTMModel

In [26]:
def train_lstm_model(model, train_loader, val_loader, epochs, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = MSELoss()
    optimizer = Adam(model.parameters(), lr=lr)
    
    best_val_loss = float("inf")
    best_model = None
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
    
    # Load best model before returning
    model.load_state_dict(best_model)
    return model

In [27]:
input_dim = dataset.samples_dim[2]
epochs = 100
learning_rate = 0.001

model = LSTMModel(input_dim=input_dim)

In [28]:
trained_model = train_lstm_model(model, train_loader, val_loader, epochs, learning_rate)

Epoch 1/100, Train Loss: 0.0706, Val Loss: 0.0257
Epoch 2/100, Train Loss: 0.0241, Val Loss: 0.0298
Epoch 3/100, Train Loss: 0.0208, Val Loss: 0.0206
Epoch 4/100, Train Loss: 0.0194, Val Loss: 0.0168
Epoch 5/100, Train Loss: 0.0172, Val Loss: 0.0293
Epoch 6/100, Train Loss: 0.0163, Val Loss: 0.0130
Epoch 7/100, Train Loss: 0.0155, Val Loss: 0.0128
Epoch 8/100, Train Loss: 0.0145, Val Loss: 0.0134
Epoch 9/100, Train Loss: 0.0140, Val Loss: 0.0131
Epoch 10/100, Train Loss: 0.0134, Val Loss: 0.0166
Epoch 11/100, Train Loss: 0.0132, Val Loss: 0.0220
Epoch 12/100, Train Loss: 0.0128, Val Loss: 0.0123
Epoch 13/100, Train Loss: 0.0128, Val Loss: 0.0127
Epoch 14/100, Train Loss: 0.0136, Val Loss: 0.0112
Epoch 15/100, Train Loss: 0.0137, Val Loss: 0.0111
Epoch 16/100, Train Loss: 0.0136, Val Loss: 0.0108
Epoch 17/100, Train Loss: 0.0133, Val Loss: 0.0114
Epoch 18/100, Train Loss: 0.0131, Val Loss: 0.0151
Epoch 19/100, Train Loss: 0.0132, Val Loss: 0.0112
Epoch 20/100, Train Loss: 0.0123, Val Lo

# Test

In [14]:
import numpy as np

In [15]:
def evaluate_test_set(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    predictions = []
    true_values = []
    
    with torch.no_grad():  # Disable gradient computation for inference
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            predictions.append(outputs.cpu().numpy())  # Store predictions
            true_values.append(y_batch.cpu().numpy())  # Store true labels
    
    # Concatenate the results into single arrays
    predictions = np.concatenate(predictions)
    true_values = np.concatenate(true_values)
    
    return predictions, true_values


In [16]:
dataset.mode = 'test'

test_loader = DataLoader(dataset, batch_size=1, shuffle=False)

# check first batch
X_batch, y_batch = next(iter(test_loader))
print("\nSample from test_loader:")
print("X_batch shape:", X_batch.shape)
print("y_batch shape:", y_batch.shape)
print("X_batch:", X_batch)


Sample from test_loader:
X_batch shape: torch.Size([1, 3, 7])
y_batch shape: torch.Size([1, 1])
X_batch: tensor([[[0.0000, 0.0000, 0.2408, 0.0580, 0.0000, 0.3123, 1.0000],
         [0.0000, 0.0000, 0.0774, 0.1976, 0.0000, 0.4061, 0.6364],
         [0.0000, 0.0000, 0.0260, 0.2823, 0.0000, 0.3159, 0.3636]]])


In [17]:
predictions, true_values = evaluate_test_set(trained_model, test_loader, device)

In [18]:
from sklearn.metrics import mean_squared_error, r2_score
rmse = np.sqrt(mean_squared_error(true_values, predictions))
r2 = r2_score(true_values, predictions)

print(f"Test RMSE: {rmse:.4f}")
print(f"Test R²: {r2:.4f}")

Test RMSE: 0.1745
Test R²: 0.3493
